# 1. ETL Pipeline for files

In [ ]:
# import packages

import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

## Filepaths and event csv files which are partitioned by date.

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

## Preparations for Apache Cassandra

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [ ]:
# the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# 2. Apache Cassandra

In [ ]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

Create Keyspace

In [ ]:
# TO-DO: Create a Keyspace 

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music
    WITH REPLICATION =
    {'class':'SimpleStrategy', 'replication_factor':1}"""
)
except Exception as e:
    print(e)

Set Keyspace

In [ ]:
# TO-DO: Set KEYSPACE to the keyspace specified above

try:
    session.set_keyspace('music')
except Exception as e:
    print(e)

# TASK 1 / Query 1: 
Return the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

Here, we partition data by sessionId and itemInSession and adding clustering columns namely artist, song to form the primary key.

In [ ]:
# create table

query = "CREATE TABLE IF NOT EXISTS song_library "
query = query + "(artist text, song text, length float, sessionId int, itemInSession int, PRIMARY KEY((sessionId, itemInSession), artist, song))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
# set up the CSV file.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_library (artist, song, length, sessionId, itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

In [2]:
## TASK/QUERY Execution: Add in the SELECT statement to verify the data was entered into the table

query = "SELECT artist, song, length FROM song_library WHERE sessionId=338 AND itemInSession=4;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

if rows:
    df = pd.DataFrame(list(rows))
    df.rename(columns={"artist": "SongArtist", "song": "SongTitle",
                      "length": "SongLength"}, 
              inplace=True)
    print(df)
else:
    print("No data found.")

No data found.


## TASK 2 / Query 2: 
Return only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

Here, we partition data by userId and sessionId and adding clustering column itemInSession to form the primary key.

In [3]:
# creating table
query = "CREATE TABLE IF NOT EXISTS song_library2 "
query = query + "(artist text, song text, firstName text, lastName text, itemInSession int, sessionId int, userId int, PRIMARY KEY((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)


# part of the code to set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_library2 (artist, song, firstName, lastName, itemInSession, sessionId, userId)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], line[1], line[4], int(line[3]), int(line[8]), int(line[10])))


query = "SELECT artist, song, firstName, lastName FROM song_library2 WHERE userId=10 AND sessionId=182;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

if rows:
    df = pd.DataFrame(list(rows))
    df.rename(columns={"artist": "SongArtist", "song": "SongTitle",
                      "firstName": "UserFirstname", "lastName": "UserFirstname"}, 
              inplace=True)
    print(df)
else:
    print("No data found.")

No data found.


# Task 3 / Query 3: 
Return every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Here, we partition data by song and adding clustering column userId to form the primary key.

In [11]:
# creating table
query = "CREATE TABLE IF NOT EXISTS song_library3 "
query = query + "(userId int, song text, firstName text, lastName text, PRIMARY KEY(song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)


# part of the code to set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_library3 (userId, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (int(line[10]), line[9], line[1], line[4]))


query = "SELECT firstName, lastName FROM song_library3 WHERE song='All Hands Against His Own';"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

if rows:
    df = pd.DataFrame(list(rows))
    df.rename(columns={"firstname": "UserFirstname", "lastname": "UserLastname"}, inplace=True)
    print(df)
else:
    print("No data found.")

   UserFirstname  UserLastname 
 0          Sara       Johnson


## Drop the tables before closing out the sessions

In [ ]:
query = "drop table if exists song_library;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table if exists song_library2;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table if exists song_library3;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

## Close the session and cluster connection

In [ ]:
session.shutdown()
cluster.shutdown()